<font size="+4">PostProcessing StochasticLagrangianSolver</font>

# Introduction

After the simulation of the StochasticLagrangianSolver is over, the openFOAM toolkit provides tools to analyze the data. 
The specialized tools for handling the lagrnagian data are provided in the sub-tookit StochasticLagrangianSolver of the openFOAM toolkit (see below for details) 


The StochasticLagrangianSolver sub-toolkit provides tools to load the lagrangian data as a [dask dataframe](https://docs.dask.org/en/stable/), either by spefying the directory of the case or, if hera datalayer (e.g the database) support was used, by specifying its name or parameters. 

After the data was loaded, it can be analyzed as a lagrangian data, or transformed to eulerian data (saved as a [xarray dataframe](https://docs.xarray.dev/en/stable/)).

Below we delineate the procedures to load the data, and analyse it as dask dataframe or as xarray dataframe. 




# Loading the data 

Loading the data is achieved throught the toolkit. 
Now, we will initialize it using the project name in the case configuration, by passing projectName=None (which is the default). 

The case configuration is 

```javascript
{
    "projectName":"EWTModel"
}
```



In [1]:
from hera import toolkitHome
from hera.utils.logging import initialize_logging,with_logger
initialize_logging(
 with_logger("hera.simulations",handlers=['console'],level="INFO",propagate=False)
)

tk = toolkitHome.getToolkit(toolkitName=toolkitHome.SIMULATIONS_OPENFOAM)

paraview not Found. Cannot execute the VTK pipeline.
paraview module is not Found!. VTK pipeline wont work
 INFO    : project.py/__init__(213) Initializing with logger EWTModel


For now, lets laded it without caching the results to the database. Hence `cache=False`

In [2]:
data = tk.stochasticLagrangian.getCaseResults("simulations/Dispersion",cache=False)

 INFO    : abstractLagrangianSolver.py/getCaseResults(1040) The simulations/Dispersion is (re)-calculated
 INFO    : abstractLagrangianSolver.py/getCaseResults(1043) Calculating the data. Trying to find the case described by: simulations/Dispersion
 INFO    : abstractLagrangianSolver.py/getCaseResults(1046) not found, trying as a directory
 INFO    : abstractLagrangianSolver.py/getCaseResults(1065) Checking if the case is single processor or multiprocessor


The file is loaded as a delayed dask. Hence, it is possible to perform querys and then load only the 
necessary data. For now, we will load all the data. 

In [3]:
loadeddata = data.compute()

Now, lets query particle 16

## Using the hera datalayer 

The hera datalayer allows the user to retrieve the data using either: 
* The name of the simulation
* The parameters of the simulation

Note that a query that returns more than 1 record will result in ValueError exception.

The usage is exactly like the usage of `tk.stochasticLagrangian.getCaseResults` that was shown above. 


# Processing the lagrangian data

Once the data was loaded, it is possible to 

In [16]:
p16 = loadeddata.query("id==16").sort_values("time")
p16.head()

,x,y,z,id,procId,globalID,mass,age,U_x,U_y,U_z,time
0,-0.005225,0.301343,0.017618,16.0,3.0,3.000000e+09,1.296680e-09,1.6889,-0.025878,0.349586,0.300931,1.0
16,0.255793,-0.015202,0.026750,16.0,3.0,3.000000e+09,1.296680e-09,2.6889,0.272111,-0.312984,-0.300000,2.0
8,-0.030813,0.318467,0.087374,16.0,3.0,3.000000e+09,1.296680e-09,3.6889,-0.270974,0.370206,0.299742,3.0
1,-0.147991,0.134711,0.122896,16.0,3.0,3.000000e+09,1.296680e-09,4.6889,-0.128508,-0.262778,-0.210259,4.0
1,-0.030408,0.300145,0.077249,16.0,3.0,3.000000e+09,1.296680e-09,5.6889,0.116097,0.225015,0.097708,5.0


# Converting to eulerian data

## Partial mesh 

The partial mesh format is a long table format that describes the concentration in a dxdydz cell (dx=dy=dz). 
That is, each row represent a single cell at a point in time with its concentration. Cells with zero concentration (i.e no particles in the cell) 
will not appear in the data. 

For example, 

<table>
    <tr>
        <td><b>x</b></td>
        <td><b>y</b></td>
        <td><b>z</b></td>
        <td><b>time</b></td>
        <td><b>C</b></td>
    </tr>
    <tr>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
    </tr>    
    <tr>
        <td>0</td>
        <td>2</td>
        <td>0</td>
        <td>0</td>
        <td>2</td>
    </tr>        
    <tr><td/><td>.</td></tr>
    <tr><td/><td>.</td></tr>
    <tr><td/><td>.</td></tr>
    
</table>

Where in this example the cell (0,1,0) at time 0 has no concentration. 

For example converting the `loadeddata` to cells of $10cm$, we use 

In [17]:
eulerian = tk.stochasticLagrangian.analysis.calcConcentrationPointWise(loadeddata,dxdydz=0.1)

In [18]:
eulerian

C
xI   yI   zI  time           
-0.5 -0.5 0.0 8.0    0.000001
              11.0   0.000001
              13.0   0.000004
              14.0   0.000001
              17.0   0.000003
...                       ...
 0.4  0.4 0.1 270.0  0.000001
              271.0  0.000004
              273.0  0.000001
              277.0  0.000001
              280.0  0.000001

[53560 rows x 1 columns]

# Converting to vtk point. 

In order to create presentation in paraFOAM, it is necessary to converty the files to .vtu format. 
To do so, we use the presentation layer of the stochasticlagrangian toolkit 

In [ ]:
tk.stochasticLagrangian.presentation.toVTU(loadeddata,outDirectory="VTK",outFile="dispersion")

Alternatively, one can use the CLI 


<div class="alert alert-block alert-success">
>> hera-openfoam stochasticLagrangian postProcess toVTK {case name} [--outputDirectory {outputDirectory}] [--overwrite] [--cloudName {cloudName}]
</div>

Where 

- {case name} can be the pat or the name of the simulation in the DB. 
- overwrite : if flag is true, remove the VTK in the directory (TODO) and also overwrite the caching the the DB. 
- cloudName : The name of the cloud to process. The default is kinematicCloud. 

<div class="alert alert-block alert-info">
<b>Example: </b> 
    
```
>> hera-openfoam stochasticLagrangian postProcess toVTK Dispersion
```